### Combine the SoupX corrected count matrices and save as adata

In [1]:
# Load required libraries
library(reticulate)
library(Matrix)

In [2]:
# directories for each of the samples
sample_dirs <- list.dirs("../raw_data/", recursive=FALSE)
sample_dirs

[1] "../raw_data//K1430-LV"               "../raw_data//K1430-LV-2nd"          
 [3] "../raw_data//K1430-LV-FACS"          "../raw_data//K1430-LV-FACS-KAPA"    
 [5] "../raw_data//K1485-LV-FACS"          "../raw_data//K1485-LV-nonFACS"      
 [7] "../raw_data//K1488-LV-FACS"          "../raw_data//K1488-LV-nonFACS"      
 [9] "../raw_data//K1545-LV-FACS"          "../raw_data//K1545-LV-run123"       
[11] "../raw_data//K1549-LV-FACS"          "../raw_data//K1549-LV-nonFACS"      
[13] "../raw_data//K1561-LV-FACS"          "../raw_data//K1570-LV-FACS"         
[15] "../raw_data//K1570-LV-nonFACS"       "../raw_data//K1584-LV-FACS"         
[17] "../raw_data//K1584-LV-FACS-e2"       "../raw_data//K1584-LV-run1n2"       
[19] "../raw_data//K1617-LV"               "../raw_data//K1617-LV-2nd"          
[21] "../raw_data//K1617-LV-FACS"          "../raw_data//K1617-LV-FACS-KAPA"    
[23] "../raw_data//K1622-LV-FACS"          "../raw_data//K1622-LV-run123"       
[25] "../raw_data//K1647-LV-FACS"          "../raw_data//K1647-LV-nonFACS"      
[27] "../raw_data//K1702-LV-RE-FACS"       "../raw_data//K1702-LV-RE-nonFACS"   
[29] "../raw_data//K1718-LV-FACS"          "../raw_data//K1718-LV-nonFACS"      
[31] "../raw_data//K1727-LV-FACS"          "../raw_data//K1727-LV-nonFACS"      
[33] "../raw_data//K1739-LV-FACS"          "../raw_data//K1739-LV-nonFACS"      
[35] "../raw_data//K1783-LV-FACS"          "../raw_data//K1783-LV-nonFACS"      
[37] "../raw_data//K1783-LV-RE-FACS"       "../raw_data//K1783-LV-RE-nonFACS"   
[39] "../raw_data//ND15755-Atr-1st"        "../raw_data//ND15755-Atr-3rd"       
[41] "../raw_data//ND15755-LV-1st-run123"  "../raw_data//ND15755-LV-2nd-run1234"
[43] "../raw_data//ND15755-RV-1st"         "../raw_data//ND15755-RV-2nd"        
[45] "../raw_data//ND15755-RV-3rd"         "../raw_data//ND15840-LV-1st-run1n2" 
[47] "../raw_data//ND15840-LV-2nd-run1n2"  "../raw_data//ND15840-LV-3rd"        
[49] "../raw_data//ND15840-RA-2nd-run1n2"  "../raw_data//ND15840-RA-3rd"        
[51] "../raw_data//ND15840-RA-4th"         "../raw_data//ND15840-RV-1st"        
[53] "../raw_data//ND15840-RV-2nd"         "../raw_data//ND15840-RV-3rd"

In [3]:
# extract the names of each of the samples
sample_names <- basename(sample_dirs)

In [4]:
# extract the paths to the SoupX corrected matrices
SoupX_count_rds_paths <- paste0(sample_dirs, "/STAR/Solo.out/GeneFullsoupX_corrected_counts.rds")

In [5]:
output_adata_dir <- "adata_from_SoupX/"
dir.create(output_adata_dir)

Warning message in dir.create(output_adata_dir):
“'adata_from_SoupX' already exists”


Iterate through each of the SoupX rds files and create a corresponding adata file in the output_adata_dir

In [6]:
num_SoupX_rds <- length(SoupX_count_rds_paths)

for (i in 1:num_SoupX_rds) {

    if (i %% 5 == 0) {
        print(i)
        flush.console()
    }
     
    sample_name <- sample_names[i]

    # load in the SoupX directory
    SoupX_count <- readRDS(SoupX_count_rds_paths[i])
    # transpose for adata format since the rows should correspond to the cells and the columns to the genes
    sparse_matrix <- t(SoupX_count)
    # Import the anndata module from Python
    anndata <- import("anndata")
    # Create an adata object, with the adata.obs as the rownames of the transposed sparse matrix 
    # and adata.var as the colnames
    adata <- anndata$AnnData(X = sparse_matrix, 
                            obs = data.frame(row.names = rownames(sparse_matrix)), 
                            var = data.frame(row.names = colnames(sparse_matrix)))
    
    # save the adata file
    output_adata_path <- paste0(output_adata_dir, sample_name, ".h5ad")
    adata$write(output_adata_path)
}

[1] 5
[1] 10
[1] 15
[1] 20
[1] 25
[1] 30
[1] 35
[1] 40
[1] 45
[1] 50
